# TODO title

# これなに?
TODO

## 使用するデータセット

前回の記事で作成したgoogle_analyticsのdatasetから、特定の商品をclickする確率を予測します

前回と同様にmagic commandを利用します(接続方法は前記事参照)

In [2]:
from google.cloud.bigquery import magics
from google.oauth2 import service_account
credentials = (service_account.Credentials.from_service_account_file('/Users/hiroaki/Downloads/scalable-data-science-b045d3ddcc4c.json'))
magics.context.credentials = credentials

In [3]:
%load_ext google.cloud.bigquery

In [5]:
PROJECT ='scalable-data-science'

In [6]:
%%bigquery --project $PROJECT 
SELECT * FROM `google_analytics.ga_dataset` LIMIT 5

,hour,browser,isMobile,country,productPrice,isClick
0,13,Chrome,False,United States,20990000,None
1,13,Chrome,False,United States,990000,None
2,7,Firefox,False,Canada,16990000,None
3,7,Firefox,False,Canada,16990000,None
4,13,Chrome,False,United States,12990000,None


# データセットの特徴を調べる

まず、`isClick`がどのようになっているか確認します

In [10]:
%%bigquery --project $PROJECT 
SELECT isClick, COUNT(1) AS numIsClick FROM `google_analytics.ga_dataset`
GROUP BY isClick

,isClick,numIsClick
0,None,90973
1,True,2034


この結果を見る限り、どうやらclickしなかった場合、は`False`ではなく、`null`が入っているようです  
今後の分析のことを考えると、1/0で二値化したほうが楽なので、変換します

In [37]:
%%bigquery --project $PROJECT 
CREATE OR REPLACE TABLE google_analytics.ga_dataset_cleansed AS
SELECT 
    *EXCEPT(isClick),
    IF(isClick, 1, 0) as isClick
FROM `google_analytics.ga_dataset`

""


In [38]:
%%bigquery --project $PROJECT 
SELECT * FROM `google_analytics.ga_dataset_cleansed` LIMIT 5

,hour,browser,isMobile,country,productPrice,isClick
0,0,Chrome,False,United States,0,0
1,0,Chrome,False,United States,0,0
2,1,Chrome,False,United States,0,0
3,1,Chrome,False,United States,0,0
4,1,Chrome,False,United States,0,0


# pandasを利用して統計的な特徴を確認

次に、pandasを利用して、データセットの概観を確認します。ただし、全部とってくると当然メモリに乗らない(乗る量ならそもそもBigQueryを使わなくてもよい)ので、`RAND()`関数で適当にsamplingしてきます

In [39]:
%%bigquery data_sampled --project $PROJECT 
-- 10%だけサンプリング
SELECT 
    * FROM `google_analytics.ga_dataset_cleansed` 
WHERE  RAND() < 0.1

概観を確認します

In [42]:
data_sampled.describe()

,hour,productPrice,isClick
count,9397.000000,9.397000e+03,9397.000000
mean,12.811429,2.311931e+07,0.020219
std,5.403000,2.582371e+07,0.140757
min,0.000000,0.000000e+00,0.000000
25%,9.000000,7.990000e+06,0.000000
50%,13.000000,1.699000e+07,0.000000
75%,17.000000,2.199000e+07,0.000000
max,23.000000,5.757000e+08,1.000000


大まかな値段の分布やclickする確率がわかります。ctrは0.02程度で、`productPrice`の分布は右に歪んでそうですね

# データの特徴を探索

## hourとisClickの関係

次に、各変数と`isClick`の関係を見ていきます。まず、`hour`から確認していきます